In [ ]:
import datetime as dt

import numpy as np
from scipy import fft

from cryosat_utils import *
%matplotlib inline

In [ ]:
subplot_size=5
sit_names = ['cs2', 'inp', 'lor', 'hir']
p_vec = (10,20,30)
date_delta0, date_delta1 = -2, 2
date0 = dt.datetime(2021,1,5)


# Prepare CS2 data

In [ ]:
cs2_files, sit_inp_name, sit_lor_name, sit_hir_name = get_filenames(date0, date_delta0, date_delta1)
odf = read_cs2_orbits(cs2_files)
plot_cs2_orbits_hist(odf)
sit_inp, sit_lor, sit_hir = get_sit_2d(sit_inp_name, sit_lor_name, sit_hir_name)
rho_ice_new, rho_snow_new  = fit_snow_ice_rho(odf, sit_inp, plot=True)
odf_masked = get_masked_odf(odf, rho_ice_new, rho_snow_new)

# Compare with CS2 data smoothed at 12 km (CNN designed output resolution)

In [ ]:
c, r, h = get_smoothed_resampled_orbits(odf_masked, smth_time='12s', resample_time='6s', min_periods=5, t0=None, t1=None)
plot_orbit_mean(c, r, h, [sit_inp, sit_lor, sit_hir])
sit_cs2 = rasterize_cs2(c, r, h, sit_inp, ws=2)

ano = {}
for arr, name in zip([sit_cs2, sit_inp, sit_lor, sit_hir], sit_names):
    print(name)
    ano[name] = compute_anomaly(arr, ws=15)

In [ ]:
titles = ['CS2', 'CS2SMOS', 'CNN-LR', 'CNN-HR']

print('FIELDS')
arrays = [sit_cs2, sit_inp, sit_lor, sit_hir]
plot_maps(arrays, titles, clim=(0, 3), subplot_size=7, step=1)

print('ANOMALY')
arrays = [ano[j][1] for j in sit_names]
plot_maps(arrays, titles, clim=(-0.2, 0), subplot_size=7, step=1)

In [ ]:
ppp = {}
for arr, name in zip([sit_cs2, sit_inp, sit_lor, sit_hir], sit_names):
    print(name)
    ppp[name] = compute_precentiles(ano[name][1], p_vec, stp=5)
    
for p in p_vec:
    print(f'ANOMALY P{p}')
    arrays = [ppp[i][p] for i in sit_names]
    plot_maps(arrays, titles, clim=(-0.2, 0), subplot_size=7, step=1)

In [ ]:
arrays = [ano[i][1] for i in sit_names]
print('Common mask')
plot_qq(arrays, titles)
print('Individual mask')
plot_qq(arrays, titles, common_mask=False)

# Find scale at which CS2 data should be smoothed

In [ ]:
arrays = [sit_inp, sit_lor, sit_hir]
titles = ['CS2SMOS', 'CNN-LR', 'CNN-HR']
plot_ks_test(odf_masked, arrays, titles)

# Compare with CS2 data smoothed at 60 km (CNN real output resolution)

In [ ]:
c2, r2, h2 = get_smoothed_resampled_orbits(odf_masked, smth_time='60s', resample_time='6s', min_periods=5, t0=None, t1=None)
plot_orbit_mean(c2, r2, h2, [sit_inp, sit_lor, sit_hir])


In [ ]:
sit_cs22 = rasterize_cs2(c2, r2, h2, sit_inp, ws=10)

sit_names2 = ['cs2', 'cs22', 'inp', 'lor', 'hir']

ano2 = {}
for arr, name in zip([sit_cs2, sit_cs22, sit_inp, sit_lor, sit_hir], sit_names2):
    print(name)
    ano2[name] = compute_anomaly(arr, ws=15)

p_vec2 = (10,20,)
ppp2 = {}
for arr, name in zip([sit_cs2, sit_cs22, sit_inp, sit_lor, sit_hir], sit_names2):
    print(name)
    ppp2[name] = compute_precentiles(ano2[name][1], p_vec2, stp=5)

In [ ]:
titles2 = ['CS2_12km', 'CS2_60km', 'CS2SMOS', 'CNN-LR', 'CNN-HR']
for p in p_vec2:
    print(f'ANOMALY P{p}')
    arrays = [ppp2[i][p] for i in sit_names2]
    plot_maps(arrays, titles2, clim=(-0.2, 0), subplot_size=7, step=1)
    
    
arrays = [ano2[i][1] for i in sit_names2]
print('Common mask')
plot_qq(arrays, titles2)
print('Individual mask')
plot_qq(arrays, titles2, common_mask=False)

In [ ]:
colors = ['b', 'g']
labels = ['CS2SMOS', 'CNN-HI-RES']
arrays = [sit_inp, sit_hir]
plot_orbit_anomaly_quantile(odf_masked, arrays, colors, labels)

# Split in chunks, compute power spectra

In [ ]:
colors = ['b', 'g']
labels = ['CS2SMOS', 'CNN-HI-RES']
arrays = [sit_inp, sit_hir]
km0=2.8485e6
km1=2.8502e6
km0=0
km1=100e6

plot_q=0.1

odf_masked_km = get_df_km(odf_masked)

gpi = (odf_masked_km > km0) * (odf_masked_km < km1) 

odf_masked2 = odf_masked[gpi]

odf_masked2_km = get_df_km(odf_masked2)

odf_m2_pix = odf_masked2.rolling('6s').mean().resample('6s').mean().dropna()
h_pix = odf_m2_pix['h'].to_numpy()
x_pix = odf_m2_pix['x'].to_numpy()
y_pix = odf_m2_pix['y'].to_numpy()

c_pix, r_pix = dst_area.get_array_coordinates_from_projection_coordinates(x_pix, y_pix)

c_pix = np.round(c_pix).astype(int)
r_pix = np.round(r_pix).astype(int)

h_inp_pix = [arr[r_pix, c_pix] for arr in arrays]

h_pix_all = np.vstack([h_pix] + h_inp_pix)

gpi_pix = np.isfinite(np.prod(h_pix_all, axis=0)).astype(int)
chunk_starts = np.where(np.diff(gpi_pix) == 1)[0]+1
chunk_ends = np.where(np.diff(gpi_pix) == -1)[0]
print(len(chunk_starts))

chunk_lengths = [che-chs for (chs,che) in zip(chunk_starts, chunk_ends)]
plt.hist(chunk_lengths, 100)
plt.show()

time_step = 6 # km
chunk_length = 40 #pix 

for hi in range(3):
    powers = []
    for chs, che in zip(chunk_starts, chunk_ends):
        sig = h_pix_all[hi][chs:che][:chunk_length]
        if sig.size < chunk_length:
            continue

        sig_fft = fft.fft(sig)
        power = np.abs(sig_fft)**2
        sample_freq = fft.fftfreq(sig.size, d=time_step)
        pos_mask = np.where(sample_freq > 0)
        freqs = sample_freq[pos_mask]
        powers.append(power[pos_mask])
    
    powers_a = 10**np.mean(np.array(np.log10(powers)), axis=0)
    plt.semilogy(1/freqs, powers_a, '.-')

plt.xlim([200, 0])
plt.xlabel('Wavelength [km]')
plt.legend(['CS2', 'CS2SMOS', 'CNN-HR'])
plt.show()